In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from common_functions import display

spark = (SparkSession.builder.remote("sc://localhost:15002").getOrCreate())

data = [[2021, "CS5302-AI621", "Albany", "M", 42]]
columns = ["Year", "First_Name", "County", "Gender", "Count"]

mydf = spark.createDataFrame(data, schema="Year int, First_Name STRING, County STRING, Gender STRING, Count int")
display(mydf)

Year,First_Name,County,Gender,Count
2021,CS5302-AI621,Albany,M,42


In [3]:
mydf.show()

+----+------------+------+------+-----+
|Year|  First_Name|County|Gender|Count|
+----+------------+------+------+-----+
|2021|CS5302-AI621|Albany|     M|   42|
+----+------------+------+------+-----+



In [8]:
nyc_df = spark.read.csv("/opt/spark/data/nyc_health_data.csv", header=True, inferSchema=True)
nyc_df.show(5)

+----+----------+------+---+-----+
|Year|First Name|County|Sex|Count|
+----+----------+------+---+-----+
|2022|    OLIVIA|Albany|  F|   16|
|2022|    AMELIA|Albany|  F|   15|
|2022|     AVERY|Albany|  F|   12|
|2022|      EMMA|Albany|  F|   11|
|2022| CHARLOTTE|Albany|  F|   11|
+----+----------+------+---+-----+
only showing top 5 rows


In [9]:
nyc_df = spark.read.csv("/opt/spark/data/nyc_health_data.csv", header=True, inferSchema=True)
nyc_df = nyc_df.withColumnRenamed("Sex", "Gender")
display(nyc_df,5)

Year,First Name,County,Gender,Count
2022,OLIVIA,Albany,F,16
2022,AMELIA,Albany,F,15
2022,AVERY,Albany,F,12
2022,EMMA,Albany,F,11
2022,CHARLOTTE,Albany,F,11


In [5]:
nyc_df.explain("formatted")

== Physical Plan ==
* Project (2)
+- Scan csv  (1)


(1) Scan csv 
Output [5]: [Year#138, First Name#139, County#140, Sex#141, Count#142]
Batched: false
Location: InMemoryFileIndex [file:/opt/spark/data/nyc_health_data.csv]
ReadSchema: struct<Year:int,First Name:string,County:string,Sex:string,Count:int>

(2) Project [codegen id : 1]
Output [5]: [Year#138, First Name#139, County#140, Sex#141 AS Gender#151, Count#142]
Input [5]: [Year#138, First Name#139, County#140, Sex#141, Count#142]




In [6]:
mydf.printSchema()
nyc_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: integer (nullable = true)

root
 |-- Year: integer (nullable = true)
 |-- First Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: integer (nullable = true)



In [10]:
# Apache Spark union() method to combine the contents of your first DataFrame mydf with DataFrame nyc_df data loaded from the CSV file.
df = mydf.union(nyc_df)
display(df,5)

Year,First_Name,County,Gender,Count
2021,CS5302-AI621,Albany,M,42
2022,OLIVIA,Albany,F,16
2022,AMELIA,Albany,F,15
2022,AVERY,Albany,F,12
2022,EMMA,Albany,F,11


In [ ]:
display(
    df.filter(df["Count"] > 50), 5
    )

In [11]:
from pyspark.sql.functions import desc
display(
    df.select("First_Name", "Count").orderBy(desc("Count")), 5)

First_Name,Count
DAVID,297
DAVID,287
ETHAN,273
DAVID,273
DAVID,272


In [12]:
subsetDF = df.filter((df["Year"] == 2009) & (df["Count"] > 100) & (df["gender"] == "F")) \
    .select("First_Name", "County", "Count") \
    .orderBy(desc("Count"))
display(subsetDF, 5)

First_Name,County,Count
LEAH,Kings,186
SARAH,Kings,183
RACHEL,Kings,170
ESTHER,Kings,168
ISABELLA,Queens,165


Spark Transformations (No Execution happens until action is executed (show, count, write, etc.)

In [13]:
# Immutability & Laziness
subset_df = df.filter(df.Count > 100) 

In [14]:
# Column Projection & Derivation
# select keeps or renames columns, withColumn creates derived columns, drop removes fields.

df_a = df.select("First_Name", "County")
df_b = df.withColumn("Count_Double", df.Count * 2).drop("Year")
display(df_b,5)

First_Name,County,Gender,Count,Count_Double
CS5302-AI621,Albany,M,42,84
OLIVIA,Albany,F,16,32
AMELIA,Albany,F,15,30
AVERY,Albany,F,12,24
EMMA,Albany,F,11,22


In [ ]:
# groupBy followed by agg computes rollups; use aliases for clarity.

df_grouped = df.groupBy("Year").agg(F.sum("Count").alias("Total_Count"))

In [ ]:
# Joins: inner, left, right, full, semi, anti

# inner join: returns only rows whose keys exist in both tables
df_inner = df.join(nyc_df, ["Year", "County"], "inner")

# right join: keeps all rows from nyc_df and matches from df where keys align
df_right = df.join(nyc_df, ["Year", "County"], "right")

# full join: keeps every row from both sides, matching where keys intersect
df_full = df.join(nyc_df, ["Year", "County"], "outer")

# left semi join: keeps rows from df that have matching keys in nyc_df without nyc_df columns
df_left_semi = df.join(nyc_df, ["Year", "County"], "leftsemi")

# left anti join: keeps rows from df whose keys do not appear in nyc_df
df_left_anti = df.join(nyc_df, ["Year", "County"], "leftanti")

# left join: keeps all rows from df and matches from nyc_df where keys align
df_left = df.join(nyc_df, ["Year", "County"], "left")

In [ ]:
# Set Operations: unionByName, intersect, and subtract combine DataFrames with aligned schemas.
# unionByName: Aligns columns by name before combining DataFrames, adding nulls for missing fields.
# intersect: Keeps only rows that exist in both DataFrames with identical column values.
# subtract: Emits rows from the first DataFrame that are absent from the second.

df_union = df.unionByName(nyc_df, allowMissingColumns=True)
df_intersect = df.intersect(nyc_df)
df_subtract = df.subtract(nyc_df)

In [ ]:
# dropDuplicates targets specific columns
# Rows sharing identical values in both First_Name and County are collapsed to their first occurrence; 
# other column values come from that retained row.

df.dropDuplicates(["First_Name", "County"])

In [ ]:
#Partition Management: repartition increases partitions for parallelism; coalesce reduces without shuffle.
df.repartition("County")

In [ ]:
# Window Transformations - window functions add rankings or moving aggregates without collapsing rows.
from pyspark.sql.window import Window

df_windows = df.withColumn("Rank", F.row_number().over(Window.partitionBy("Year").orderBy(desc("Count"))))
display(df_windows,5)

In [ ]:
# Caching & Checkpoints - cache/persist reuse expensive intermediate plans; checkpoint breaks lineage for stability.
df.cache()

In [ ]:
# Diagnostics: explain reveals the logical/physical plan; use to spot wide shuffles or scans.
df.filter(df.Count > 100).explain()
